In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/yahoo-finance-stock-market-data/pre_processed_data.csv
/kaggle/input/yahoo-finance-stock-market-data/stock_market_data.csv


In [2]:
!pip install pyspark
!pip install pmdarima
!pip install darts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ | done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488512 sha256=3900f07cb36bfc2b035cf76f9fe7950d2b58f1cf65ef9ad7ac7170ce33062994
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 884.7/884.7 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.5/160.5 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, when
from pyspark.sql.window import Window
import datetime
import warnings
warnings.simplefilter("ignore")

# from datetime import datetime
sc = SparkContext('local')


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/30 16:14:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark = SparkSession(sc)
df=spark.read.format("csv").option("header","true").load('/kaggle/input/yahoo-finance-stock-market-data/stock_market_data.csv')
df.show()

+----------+---------+---------+---------+---------+
|      Date|     AMZN|     GOOG|     QCOM|     AMAT|
+----------+---------+---------+---------+---------+
|02/01/2020|94.900497|  68.3685|88.690002|62.200001|
|03/01/2020|93.748497|68.032997|87.019997|61.209999|
|06/01/2020|95.143997|69.710503|86.510002|59.889999|
|07/01/2020|95.343002|   69.667|88.970001|61.619999|
|08/01/2020|94.598503|70.216003|88.709999|61.580002|
|09/01/2020|95.052498|70.991501|89.910004|61.970001|
|10/01/2020|94.157997|71.486504|90.260002|61.470001|
|13/01/2020|94.565002|71.961502|90.970001|61.810001|
|14/01/2020|   93.472|71.543999|90.559998|62.349998|
|15/01/2020|93.100998|71.959999|89.669998|    61.93|
|16/01/2020|93.897003|72.584999|91.790001|63.209999|
|17/01/2020|   93.236|74.019501|95.910004|62.849998|
|21/01/2020|94.599998|74.220001|94.540001|63.200001|
|22/01/2020|94.373001|74.297501|92.970001|63.889999|
|23/01/2020|94.228996|74.332497|       92|63.889999|
|24/01/2020|93.082001|73.335503|89.650002|    

In [5]:
# warnings.simplefilter("ignore")
# df = df.withColumn("Date", F.to_date(F.col("Date"), 'dd/MM/yyyy'))
# min_date = df.select(F.min("Date")).first()[0]
# max_date = df.select(F.max("Date")).first()[0]
# date_range = [min_date + datetime.timedelta(days=x) for x in range((max_date - min_date).days + 1)]
# all_dates_df = spark.createDataFrame([(date,) for date in date_range], ["Date"])
# full_df = all_dates_df.join(df, on="Date", how="left_outer").orderBy("Date")
# window = (Window.orderBy('Date').rowsBetween(Window.unboundedPreceding, Window.currentRow))
# full_df = (full_df.withColumn('AMZN', F.last('AMZN', ignorenulls=True).over(window)))
# full_df = (full_df.withColumn('GOOG', F.last('GOOG', ignorenulls=True).over(window)))
# full_df = (full_df.withColumn('QCOM', F.last('QCOM', ignorenulls=True).over(window)))
# full_df = (full_df.withColumn('AMAT', F.last('AMAT', ignorenulls=True).over(window)))
    
# full_df.show()


In [6]:
# full_df.toPandas().to_csv('pre_processed_data.csv',header=True)
spark.stop()

In [7]:
stock_data = pd.read_csv('/kaggle/input/yahoo-finance-stock-market-data/pre_processed_data.csv')
stock_data = stock_data.drop(columns=['Unnamed: 0'])
stock_data.head()

,Date,AMZN,GOOG,QCOM,AMAT
0,2020-01-02,94.900497,68.368500,88.690002,62.200001
1,2020-01-03,93.748497,68.032997,87.019997,61.209999
2,2020-01-04,93.748497,68.032997,87.019997,61.209999
3,2020-01-05,93.748497,68.032997,87.019997,61.209999
4,2020-01-06,95.143997,69.710503,86.510002,59.889999


**THE SHARE PRICES ARE IN POUNDS AND NOT IN DOLLARS**

In [8]:
import plotly.express as px
fig = px.line(stock_data, x='Date', y=["GOOG","AMZN","QCOM","AMAT"],title='Stock market data')
fig.show()

In [9]:
stock_data['Date'] = pd.to_datetime(stock_data['Date'])

# Set 'Date' as the index
stock_data.set_index('Date', inplace=True)

# Resample data bi-weekly and calculate the mean for each group
_12_weekly_averages = stock_data.resample('12W').mean()

print('Length of Original data',len(stock_data))
print('Length of 12 weekly data',len(_12_weekly_averages))
# Display the first few rows of the bi-weekly average data
_12_weekly_averages.head()

Length of Original data 1433
Length of 12 weekly data 19


,AMZN,GOOG,QCOM,AMAT
Date,,,,
2020-01-05,94.036497,68.116873,87.437498,61.457500
2020-03-29,96.915993,68.315031,82.433572,57.720595
2020-06-21,118.013739,66.755420,78.582500,52.992381
2020-09-13,155.815135,76.173965,103.156309,62.253572
2020-12-06,158.086309,81.194404,131.074286,66.867262


In [10]:
_12_weekly_averages_df = _12_weekly_averages.reset_index()
fig = px.line(_12_weekly_averages_df, x='Date', y=["GOOG","AMZN","QCOM","AMAT"],title='Stock market data')
fig.show()

# **Autoregressive (AR) model**
An Autoregressive (AR) model is a type of time series forecasting model that uses a linear combination of the past values of the variable to predict future values. The model can be represented as:


Xt
​
 =c+ϕ 
1
​
 Xt−1
​
 +ϕ 
2
​
 Xt−2
​
 +…+ϕ 
p
​
 Xt−p
​
 +ϵt
​
 

Here, 

Xt
​
is the variable at time 

t,c is a constant, 

ϕ1
​
 ,ϕ2
​
 ,…,ϕp are parameters of the model, and ϵt is white noise.

In [11]:
import numpy as np
import plotly.graph_objects as go
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA

# Function to fit AR model and forecast the next period
def forecast_ar(stock_data):
    # Fit an AR model
    model = ARIMA(stock_data, order=(1,0,0))  # Equivalent to AR(1) model
    model_fit = model.fit()
    # Forecast the next bi-weekly period
    forecast = model_fit.forecast(steps=1)
    
    # Fit an AR model
    model = ARIMA(stock_data[:-1], order=(1,0,0))  # Equivalent to AR(1) model
    model_fit = model.fit()
    # Forecast the next bi-weekly period
    last_forecast = model_fit.forecast(steps=1)
    
    return model_fit, forecast,last_forecast[0],stock_data[-1]

# Dictionary to store models and forecasts
models_last_forecasts = {}
models_future_forecasts = {}
df_results=pd.DataFrame(columns=['Model','Company','Actual','Forecasted','Error'])
# Loop through each stock column and apply the forecasting function
for column in _12_weekly_averages.columns:
    fit, forecast,last_forecast,actual = forecast_ar(_12_weekly_averages[column])
    models_future_forecasts[column] = (fit, forecast)
    models_last_forecasts[column]=(fit,[actual,last_forecast,actual-last_forecast])
    
    df= pd.DataFrame([['AR',column,actual,last_forecast,actual-last_forecast]],columns=['Model','Company','Actual','Forecasted','Error'])
    df_results=pd.concat([df_results,df])
    df_results=df_results.reset_index(drop=True)
# # Display the forecasted last values for each stock   
# print({key: val[1] for key, val in models_last_forecasts.items()})
# Display the forecasted future values for each stock
print('prediction of future date')
print({key: val[1] for key, val in models_future_forecasts.items()})
df_results.head()

prediction of future date
{'AMZN': 2024-05-19    100.588784
Freq: 12W-SUN, dtype: float64, 'GOOG': 2024-05-19    104.14976
Freq: 12W-SUN, dtype: float64, 'QCOM': 2024-05-19    120.062339
Freq: 12W-SUN, dtype: float64, 'AMAT': 2024-05-19    111.050766
Freq: 12W-SUN, dtype: float64}


,Model,Company,Actual,Forecasted,Error
0,AR,AMZN,97.830002,101.222115,-3.392113
1,AR,GOOG,105.220001,100.845243,4.374758
2,AR,QCOM,120.269997,121.372908,-1.102911
3,AR,AMAT,113.160004,113.510634,-0.350630


In [12]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# Assuming '_12_weekly_averages' is your DataFrame
# Create subplots
fig_comparison = make_subplots(rows=2, cols=2, subplot_titles=("Amazon (AMZN)", "Google (GOOG)", "Qualcomm (QCOM)", "Applied Materials (AMAT)"))

def plot_ar_forecast_comparison(row, col, stock_name):
    # Fit AR(1) model excluding the last bi-weekly data
    model = ARIMA(_12_weekly_averages[stock_name][:-1], order=(1,0,0))
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=1)
    forecast_index = _12_weekly_averages.index[-1]
    forecast_value = forecast[0]
    
    # Plot historical data
    fig_comparison.add_trace(go.Scatter(x=_12_weekly_averages.index, y=_12_weekly_averages[stock_name], mode='lines', name=f'{stock_name} - Historical'), row=row, col=col)
    
    # Add forecast point
    fig_comparison.add_trace(go.Scatter(x=[forecast_index], y=[forecast_value], mode='markers', name=f'{stock_name} - Forecast', marker=dict(color='red', size=10)), row=row, col=col)
    
    # Draw a vertical line at the forecast point
    fig_comparison.add_vline(x=forecast_index, line_dash="dash", line_color="green", row=row, col=col)

plot_ar_forecast_comparison(1, 1, 'AMZN')
plot_ar_forecast_comparison(1, 2, 'GOOG')
plot_ar_forecast_comparison(2, 1, 'QCOM')
plot_ar_forecast_comparison(2, 2, 'AMAT')

fig_comparison.update_layout(title_text="AR MODEL Historical and Predicted Last 12-Weekly Stock Values", height=800)
fig_comparison.show()


# **Moving Average (MA)**
A Moving Average (MA) model in time series forecasting is a method that models the next time step as a function of the past error terms. The formula for an MA model of order \( q \) is given by:

\[ X_t = \mu + \epsilon_t + \theta_1 \epsilon_{t-1} + \theta_2 \epsilon_{t-2} + \ldots + \theta_q \epsilon_{t-q} \]

where \( \mu \) is the mean of the series, \( \epsilon_t \) is white noise, and \( \theta_1, \theta_2, \ldots, \theta_q \) are the parameters of the model. This approach is particularly useful for modeling time series that show short-term dependencies between consecutive observations.

In [13]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# Function to fit MA model and forecast the next period
def forecast_ma(stock_data):
    # Fit an MA model for future prediction
    model_future = ARIMA(stock_data, order=(0,0,1))  # Equivalent to MA(1) model
    model_future_fit = model_future.fit()
    future_forecast = model_future_fit.forecast(steps=1)

    # Fit an MA model excluding the last data point for last period prediction
    model_last = ARIMA(stock_data[:-1], order=(0,0,1))  # Equivalent to MA(1) model
    model_last_fit = model_last.fit()
    last_forecast = model_last_fit.forecast(steps=1)

    actual = stock_data[-1]
    return model_future_fit, future_forecast, last_forecast[0], actual

# Dictionary to store models and forecasts
models_last_forecasts = {}
models_future_forecasts = {}

# Loop through each stock column and apply the forecasting function
for column in _12_weekly_averages.columns:
    fit, future_forecast, last_forecast, actual = forecast_ma(_12_weekly_averages[column])
    models_future_forecasts[column] = (fit, future_forecast)
    models_last_forecasts[column] = (fit, [actual, last_forecast, actual - last_forecast])
    
    df = pd.DataFrame([['MA', column, actual, last_forecast, actual - last_forecast]], columns=['Model', 'Company', 'Actual', 'Forecasted', 'Error'])
    df_results = pd.concat([df_results, df])
    
df_results = df_results.drop_duplicates()
df_results = df_results.reset_index(drop=True)
# Display the forecasted future values for each stock
print('Prediction of future date')
print({key: val[1] for key, val in models_future_forecasts.items()})

df_results[df_results['Model']=='MA'].head()


Prediction of future date
{'AMZN': 2024-05-19    116.576678
Freq: 12W-SUN, dtype: float64, 'GOOG': 2024-05-19    107.374032
Freq: 12W-SUN, dtype: float64, 'QCOM': 2024-05-19    121.866776
Freq: 12W-SUN, dtype: float64, 'AMAT': 2024-05-19    106.55401
Freq: 12W-SUN, dtype: float64}


,Model,Company,Actual,Forecasted,Error
4,MA,AMZN,97.830002,114.973522,-17.143520
5,MA,GOOG,105.220001,97.362835,7.857166
6,MA,QCOM,120.269997,123.432427,-3.162430
7,MA,AMAT,113.160004,106.906355,6.253649


In [14]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# Assuming '_12_weekly_averages' is your DataFrame
# Create subplots
fig_comparison = make_subplots(rows=2, cols=2, subplot_titles=("Amazon (AMZN)", "Google (GOOG)", "Qualcomm (QCOM)", "Applied Materials (AMAT)"))

def plot_ma_forecast_comparison(row, col, stock_name):
    # Fit MA(1) model excluding the last 12-weekly data
    model = ARIMA(_12_weekly_averages[stock_name][:-1], order=(0,0,1))
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=1)
    forecast_index = _12_weekly_averages.index[-1]
    forecast_value = forecast[0]
    
    # Plot historical data
    fig_comparison.add_trace(go.Scatter(x=_12_weekly_averages.index, y=_12_weekly_averages[stock_name], mode='lines', name=f'{stock_name} - Historical'), row=row, col=col)
    
    # Add forecast point
    fig_comparison.add_trace(go.Scatter(x=[forecast_index], y=[forecast_value], mode='markers', name=f'{stock_name} - Forecast', marker=dict(color='red', size=10)), row=row, col=col)
    
    # Draw a vertical line at the forecast point
    fig_comparison.add_vline(x=forecast_index, line_dash="dash", line_color="green", row=row, col=col)

plot_ma_forecast_comparison(1, 1, 'AMZN')
plot_ma_forecast_comparison(1, 2, 'GOOG')
plot_ma_forecast_comparison(2, 1, 'QCOM')
plot_ma_forecast_comparison(2, 2, 'AMAT')

fig_comparison.update_layout(title_text="MA MODEL Historical and Predicted Last 12-Weekly Stock Values", height=800)
fig_comparison.show()


# **ARIMA (AutoRegressive Integrated Moving Average)**
An ARIMA (AutoRegressive Integrated Moving Average) model is used for forecasting time series data by combining differencing (to make data stationary), autoregression (AR), and moving average (MA) components. The formula for an ARIMA model is generally expressed as:

\[ \nabla^d X_t = c + \phi_1 \nabla^d X_{t-1} + \dots + \phi_p \nabla^d X_{t-p} + \theta_1 \epsilon_{t-1} + \dots + \theta_q \epsilon_{t-q} + \epsilon_t \]

where \( \nabla^d X_t \) represents the differenced series, \( p \) and \( q \) are the orders of the AR and MA parts, \( d \) is the degree of differencing, \( \phi \) and \( \theta \) are parameters, and \( \epsilon_t \) is white noise. This model can capture complex patterns in time series data, making it effective for a wide range of applications.

In [15]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# Function to fit ARIMA model and forecast the next period
def forecast_arima(stock_data):
    # Fit an ARIMA model for future prediction
    model_future = ARIMA(stock_data, order=(1,1,1))  # ARIMA(1,1,1) model
    model_future_fit = model_future.fit()
    future_forecast = model_future_fit.forecast(steps=1)

    # Fit an ARIMA model excluding the last data point for last period prediction
    model_last = ARIMA(stock_data[:-1], order=(1,1,1))  # ARIMA(1,1,1) model
    model_last_fit = model_last.fit()
    last_forecast = model_last_fit.forecast(steps=1)

    actual = stock_data[-1]
    return model_future_fit, future_forecast, last_forecast[0], actual

# Dictionary to store models and forecasts
models_last_forecasts = {}
models_future_forecasts = {}

# Assuming '_12_weekly_averages' is your DataFrame
# Loop through each stock column and apply the forecasting function
for column in _12_weekly_averages.columns:
    fit, future_forecast, last_forecast, actual = forecast_arima(_12_weekly_averages[column])
    models_future_forecasts[column] = (fit, future_forecast)
    models_last_forecasts[column] = (fit, [actual, last_forecast, actual - last_forecast])
    
    df = pd.DataFrame([['ARIMA', column, actual, last_forecast, actual - last_forecast]], columns=['Model', 'Company', 'Actual', 'Forecasted', 'Error'])
    df_results = pd.concat([df_results, df])

df_results = df_results.drop_duplicates()
df_results = df_results.reset_index(drop=True)

# Display the forecasted future values for each stock
print('Prediction of future date')
print({key: val[1] for key, val in models_future_forecasts.items()})

df_results[df_results['Model']=='ARIMA'].head()


Prediction of future date
{'AMZN': 2024-05-19    97.517552
Freq: 12W-SUN, dtype: float64, 'GOOG': 2024-05-19    107.00652
Freq: 12W-SUN, dtype: float64, 'QCOM': 2024-05-19    120.227022
Freq: 12W-SUN, dtype: float64, 'AMAT': 2024-05-19    111.899237
Freq: 12W-SUN, dtype: float64}


,Model,Company,Actual,Forecasted,Error
8,ARIMA,AMZN,97.830002,97.797648,0.032354
9,ARIMA,GOOG,105.220001,103.297876,1.922125
10,ARIMA,QCOM,120.269997,121.961419,-1.691422
11,ARIMA,AMAT,113.160004,115.514663,-2.354659


In [16]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# Assuming '_12_weekly_averages' is your DataFrame
# Create subplots
fig_comparison = make_subplots(rows=2, cols=2, subplot_titles=("Amazon (AMZN)", "Google (GOOG)", "Qualcomm (QCOM)", "Applied Materials (AMAT)"))

def plot_arima_forecast_comparison(row, col, stock_name):
    # Fit ARIMA(1,1,1) model excluding the last 12-weekly data
    model = ARIMA(_12_weekly_averages[stock_name][:-1], order=(1,1,1))
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=1)
    forecast_index = _12_weekly_averages.index[-1]
    forecast_value = forecast[0]
    
    # Plot historical data
    fig_comparison.add_trace(go.Scatter(x=_12_weekly_averages.index, y=_12_weekly_averages[stock_name], mode='lines', name=f'{stock_name} - Historical'), row=row, col=col)
    
    # Add forecast point
    fig_comparison.add_trace(go.Scatter(x=[forecast_index], y=[forecast_value], mode='markers', name=f'{stock_name} - Forecast', marker=dict(color='red', size=10)), row=row, col=col)
    
    # Draw a vertical line at the forecast point
    fig_comparison.add_vline(x=forecast_index, line_dash="dash", line_color="green", row=row, col=col)

plot_arima_forecast_comparison(1, 1, 'AMZN')
plot_arima_forecast_comparison(1, 2, 'GOOG')
plot_arima_forecast_comparison(2, 1, 'QCOM')
plot_arima_forecast_comparison(2, 2, 'AMAT')

fig_comparison.update_layout(title_text="ARIMA(1,1,1) MODEL Historical and Predicted Last 12-Weekly Stock Values", height=800)
fig_comparison.show()


# **SARIMA (Seasonal ARIMA)**
SARIMA (Seasonal ARIMA) extends the ARIMA model to account for seasonality in time series data. The model includes both non-seasonal and seasonal elements:
(1−ϕ 
1
​
 B−⋯−ϕ 
p
​
 B 
p
 )(1−Φ 
1
​
 B 
s
 −⋯−Φ 
P
​
 B 
sP
 )(1−B) 
d
 (1−B 
s
 ) 
D
 X 
t
​
 =(1+θ 
1
​
 B+⋯+θ 
q
​
 B 
q
 )(1+Θ 
1
​
 B 
s
 +⋯+Θ 
Q
​
 B 
sQ
 )ϵ 
t
​


Here, 
B is the backshift operator, 
d and 
D are the orders of regular and seasonal differencing, 
p, 
q, 
P, and 
Q are the orders of the autoregressive and moving average terms for the non-seasonal and seasonal parts, and 
s is the length of the seasonal cycle.

In [17]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Function to fit SARIMA model and forecast the next period
def forecast_sarima(stock_data):
    # Fit a SARIMA model for future prediction
    model_future = SARIMAX(stock_data, order=(1,1,1), seasonal_order=(1,1,1,12))  # SARIMA(1,1,1)(1,1,1)[12]
    model_future_fit = model_future.fit()
    future_forecast = model_future_fit.forecast(steps=1)

    # Fit a SARIMA model excluding the last data point for last period prediction
    model_last = SARIMAX(stock_data[:-1], order=(1,1,1), seasonal_order=(1,1,1,12))  # SARIMA(1,1,1)(1,1,1)[12]
    model_last_fit = model_last.fit()
    last_forecast = model_last_fit.forecast(steps=1)

    actual = stock_data[-1]
    return model_future_fit, future_forecast, last_forecast[0], actual

# Dictionary to store models and forecasts
models_last_forecasts = {}
models_future_forecasts = {}

# Assuming '_12_weekly_averages' is your DataFrame
# Loop through each stock column and apply the forecasting function
for column in _12_weekly_averages.columns:
    fit, future_forecast, last_forecast, actual = forecast_sarima(_12_weekly_averages[column])
    models_future_forecasts[column] = (fit, future_forecast)
    models_last_forecasts[column] = (fit, [actual, last_forecast, actual - last_forecast])
    
    df = pd.DataFrame([['SARIMA', column, actual, last_forecast, actual - last_forecast]], columns=['Model', 'Company', 'Actual', 'Forecasted', 'Error'])
    df_results = pd.concat([df_results, df])

df_results = df_results.reset_index(drop=True)

# Display the forecasted future values for each stock
print('Prediction of future date')
print({key: val[1] for key, val in models_future_forecasts.items()})

df_results[df_results['Model']=='SARIMA'].head()


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.28057D+01    |proj g|=  4.50754D+01

At iterate    5    f=  3.79344D+00    |proj g|=  1.42657D+00

At iterate   10    f=  1.36338D+00    |proj g|=  1.23574D-02

At iterate   15    f=  1.36305D+00    |proj g|=  1.25889D-04

At iterate   20    f=  1.36304D+00    |proj g|=  2.51861D-03


 This problem is unconstrained.



At iterate   25    f=  1.36289D+00    |proj g|=  2.53822D-03

At iterate   30    f=  1.36243D+00    |proj g|=  4.87498D-03

At iterate   35    f=  1.36078D+00    |proj g|=  3.93732D-03

At iterate   40    f=  1.36021D+00    |proj g|=  2.75897D-04

At iterate   45    f=  1.36003D+00    |proj g|=  5.93957D-04

At iterate   50    f=  1.36000D+00    |proj g|=  3.71215D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     50     57      1     0     0   3.712D-04   1.360D+00
  F =   1.3599967336280381     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 
RUNNING THE L-BFGS-B CODE

           * * *

Machin

/opt/conda/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

 This problem is unconstrained.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5      2     27      1     0     0   1.993D+09   4.096D+06
  F =   4095604.1732496046     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.29374D+01    |proj g|=  1.38837D+01

At iterate    5    f=  1.23562D+00    |proj g|=  7.84311D-02

At iterate   10    f=  1.22087D+00    |proj g|=  1.24466D-02

At iterate   15    f=  1.21553D+00    |proj g|=  2.23239D-03


...

/opt/conda/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

 This problem is unconstrained.



At iterate   50    f=  1.35916D+00    |proj g|=  4.70107D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     50     65      1     0     0   4.701D-04   1.359D+00
  F =   1.3591609758805674     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.53898D+11    |proj g|=  1.52473D+16



 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.



At iterate    5    f=  2.17649D+00    |proj g|=  4.62388D-01

At iterate   10    f=  1.19935D+00    |proj g|=  1.48172D-02

At iterate   15    f=  1.19825D+00    |proj g|=  1.31652D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     19     70      2     0     0   4.570D-06   1.198D+00
  F =   1.1982212887840946     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
Prediction of future date
{'AMZN': 2024-05-19    107.07968
Freq: 12W-SUN, dtype: float64, 'GOOG': 2024-05-19    118.185664
Freq: 12W-SUN, dtype: float64, 'QCOM': 2024-05-19    136.285095
Freq: 12W-SUN, dtype: float64, 'AMAT': 2024-05-19 

,Model,Company,Actual,Forecasted,Error
12,SARIMA,AMZN,97.830002,98.360939,-0.530937
13,SARIMA,GOOG,105.220001,118.433662,-13.213661
14,SARIMA,QCOM,120.269997,89.036852,31.233145
15,SARIMA,AMAT,113.160004,138.998331,-25.838327


In [18]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Assuming '_12_weekly_averages' is your DataFrame
# Create subplots
fig_comparison = make_subplots(rows=2, cols=2, subplot_titles=("Amazon (AMZN)", "Google (GOOG)", "Qualcomm (QCOM)", "Applied Materials (AMAT)"))

def plot_sarima_forecast_comparison(row, col, stock_name):
    # Fit SARIMA model excluding the last 12-weekly data
    model = SARIMAX(_12_weekly_averages[stock_name][:-1], order=(1,1,1), seasonal_order=(1,1,1,12))  # SARIMA(1,1,1)(1,1,1)[12] model
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=1)
    forecast_index = _12_weekly_averages.index[-1]
    forecast_value = forecast[0]
    
    # Plot historical data
    fig_comparison.add_trace(go.Scatter(x=_12_weekly_averages.index, y=_12_weekly_averages[stock_name], mode='lines', name=f'{stock_name} - Historical'), row=row, col=col)
    
    # Add forecast point
    fig_comparison.add_trace(go.Scatter(x=[forecast_index], y=[forecast_value], mode='markers', name=f'{stock_name} - Forecast', marker=dict(color='red', size=10)), row=row, col=col)
    
    # Draw a vertical line at the forecast point
    fig_comparison.add_vline(x=forecast_index, line_dash="dash", line_color="green", row=row, col=col)

plot_sarima_forecast_comparison(1, 1, 'AMZN')
plot_sarima_forecast_comparison(1, 2, 'GOOG')
plot_sarima_forecast_comparison(2, 1, 'QCOM')
plot_sarima_forecast_comparison(2, 2, 'AMAT')

fig_comparison.update_layout(title_text="SARIMA(1,1,1)(1,1,1)[12] MODEL Historical and Predicted Last 12-Weekly Stock Values", height=800)
fig_comparison.show()


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.65049D+12    |proj g|=  6.49958D+17


 This problem is unconstrained.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5      2     27      1     0     0   1.993D+09   4.096D+06
  F =   4095604.1732496046     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.57433D+11    |proj g|=  3.14669D+16



   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.
 This problem is unconstrained.

   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.
 This problem is unconstrained.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5      2     19      1     0     0   2.147D+07   1.321D+05
  F =   132086.88013998087     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  7.12120D+11    |proj g|=  4.80926D+16



 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.



At iterate    5    f=  2.61117D+00    |proj g|=  9.45084D-01

At iterate   10    f=  1.23755D+00    |proj g|=  7.15309D-03

At iterate   15    f=  1.17471D+00    |proj g|=  3.62607D-02

At iterate   20    f=  1.16433D+00    |proj g|=  3.43170D-03

At iterate   25    f=  1.16109D+00    |proj g|=  4.77939D-02

At iterate   30    f=  1.14542D+00    |proj g|=  3.58837D-03

At iterate   35    f=  1.14390D+00    |proj g|=  4.42089D-03

At iterate   40    f=  1.13936D+00    |proj g|=  9.76533D-03

At iterate   45    f=  1.13902D+00    |proj g|=  7.78023D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     47    116   

 This problem is unconstrained.

 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.



At iterate    5    f=  2.17649D+00    |proj g|=  4.62388D-01

At iterate   10    f=  1.19935D+00    |proj g|=  1.48172D-02

At iterate   15    f=  1.19825D+00    |proj g|=  1.31652D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     19     70      2     0     0   4.570D-06   1.198D+00
  F =   1.1982212887840946     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


In [19]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
from pmdarima.arima import auto_arima

# Function to fit auto_arima model and forecast the next period
def forecast_auto_arima(stock_data):
    # Fit an auto_arima model for future prediction
    model_future = auto_arima(stock_data, seasonal=True, m=1, trace=False,
                              error_action='ignore', suppress_warnings=True,
                              stepwise=True)
    future_forecast = model_future.predict(n_periods=1)

    # Fit an auto_arima model excluding the last data point for last period prediction
    model_last = auto_arima(stock_data[:-1], seasonal=True, m=1, trace=False,
                            error_action='ignore', suppress_warnings=True,
                            stepwise=True)
    last_forecast = model_last.predict(n_periods=1)

    actual = stock_data[-1]
    return model_future, future_forecast[0], last_forecast[0], actual

# Dictionary to store models and forecasts
models_last_forecasts = {}
models_future_forecasts = {}

# Assuming '_12_weekly_averages' is your DataFrame
# Loop through each stock column and apply the forecasting function
for column in _12_weekly_averages.columns:
    fit, future_forecast, last_forecast, actual = forecast_auto_arima(_12_weekly_averages[column])
    models_future_forecasts[column] = (fit, future_forecast)
    models_last_forecasts[column] = (fit, [actual, last_forecast, actual - last_forecast])
    
    df = pd.DataFrame([['Auto-ARIMA', column, actual, last_forecast, actual - last_forecast]], columns=['Model', 'Company', 'Actual', 'Forecasted', 'Error'])
    df_results = pd.concat([df_results, df])

df_results = df_results.drop_duplicates()
df_results = df_results.reset_index(drop=True)

# Display the forecasted future values for each stock
print('Prediction of future date')
print({key: val[1] for key, val in models_future_forecasts.items()})

df_results[df_results['Model']=='Auto-ARIMA'].head()


Prediction of future date
{'AMZN': 114.39266712750407, 'GOOG': 108.62521751995833, 'QCOM': 120.06243822197764, 'AMAT': 109.08677868747418}


,Model,Company,Actual,Forecasted,Error
16,Auto-ARIMA,AMZN,97.830002,101.221458,-3.391456
17,Auto-ARIMA,GOOG,105.220001,100.371322,4.848679
18,Auto-ARIMA,QCOM,120.269997,121.372899,-1.102902
19,Auto-ARIMA,AMAT,113.160004,112.366741,0.793263


In [20]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from pmdarima.arima import auto_arima

# Create subplots
fig_comparison = make_subplots(rows=2, cols=2, subplot_titles=("Amazon (AMZN)", "Google (GOOG)", "Qualcomm (QCOM)", "Applied Materials (AMAT)"))

def plot_auto_arima_forecast_comparison(row, col, stock_name):
    # Fit auto_arima model excluding the last 12-weekly data
    model = auto_arima(_12_weekly_averages[stock_name][:-1], seasonal=True, m=1, trace=False,
                       error_action='ignore', suppress_warnings=True, stepwise=True)
    forecast = model.predict(n_periods=1)
    forecast_index = _12_weekly_averages.index[-1]
    forecast_value = forecast[0]
    
    # Plot historical data
    fig_comparison.add_trace(go.Scatter(x=_12_weekly_averages.index, y=_12_weekly_averages[stock_name], mode='lines', name=f'{stock_name} - Historical'), row=row, col=col)
    
    # Add forecast point
    fig_comparison.add_trace(go.Scatter(x=[forecast_index], y=[forecast_value], mode='markers', name=f'{stock_name} - Forecast', marker=dict(color='red', size=10)), row=row, col=col)
    
    # Draw a vertical line at the forecast point
    fig_comparison.add_vline(x=forecast_index, line_dash="dash", line_color="green", row=row, col=col)

plot_auto_arima_forecast_comparison(1, 1, 'AMZN')
plot_auto_arima_forecast_comparison(1, 2, 'GOOG')
plot_auto_arima_forecast_comparison(2, 1, 'QCOM')
plot_auto_arima_forecast_comparison(2, 2, 'AMAT')

fig_comparison.update_layout(title_text="Auto-ARIMA Model Historical and Predicted Last 12-Weekly Stock Values", height=800)
fig_comparison.show()


In [21]:
from darts import TimeSeries
from darts.models import ExponentialSmoothing

def forecast_auto_ets(stock_data, seasonal_period=4):
    # Convert to numpy array if needed
    values = stock_data.values if isinstance(stock_data, pd.Series) else np.array(stock_data)

    # Create a TimeSeries object from the values
    series = TimeSeries.from_values(values)

    # Check data length against twice the seasonal period for seasonal modeling
    if len(values) >= 2 * seasonal_period:
        # Enough data for seasonal modeling
        model = ExponentialSmoothing(seasonal_periods=seasonal_period)
        model_last = ExponentialSmoothing(seasonal_periods=seasonal_period)
    else:
        # Not enough data for seasonal modeling, use non-seasonal model
        print(f"Not enough data for seasonal modeling for series with length {len(values)} and seasonal period {seasonal_period}.")
        model = ExponentialSmoothing(seasonal_periods=None)
        model_last = ExponentialSmoothing(seasonal_periods=None)

    # Fit the models and make predictions
    model.fit(series)
    future_forecast = model.predict(n=1)
    
    # For last period prediction, create a TimeSeries object from all but the last value
    series_last = TimeSeries.from_values(values[:-1])
    model_last.fit(series_last)
    last_forecast = model_last.predict(n=1)

    actual = values[-1]
    return future_forecast.values()[0][0], last_forecast.values()[0][0], actual

# Example usage assuming '_12_weekly_averages' is a DataFrame with each column as a time series
models_future_forecasts = {}
models_last_forecasts = {}

for column in _12_weekly_averages.columns:
    future_forecast, last_forecast, actual = forecast_auto_ets(_12_weekly_averages[column])
    models_future_forecasts[column] = future_forecast
    models_last_forecasts[column] = [actual, last_forecast, actual - last_forecast]

    df = pd.DataFrame([['AutoETS', column, actual, last_forecast, actual - last_forecast]], columns=['Model', 'Company', 'Actual', 'Forecasted', 'Error'])
    df_results = pd.concat([df_results, df])

df_results = df_results.drop_duplicates().reset_index(drop=True)

# Display the forecasted future values for each stock
print('Prediction of future date')
print({key: val for key, val in models_future_forecasts.items()})
df_results[df_results['Model']=='AutoETS'].head()

Prediction of future date
{'AMZN': 99.69845498787642, 'GOOG': 107.36316286708318, 'QCOM': 123.53807081436153, 'AMAT': 103.1153638360253}


,Model,Company,Actual,Forecasted,Error
20,AutoETS,AMZN,97.830002,101.667354,-3.837352
21,AutoETS,GOOG,105.220001,105.741093,-0.521092
22,AutoETS,QCOM,120.269997,111.198804,9.071193
23,AutoETS,AMAT,113.160004,110.651414,2.508590


In [22]:
import numpy as np
import pandas as pd
from darts import TimeSeries
from darts.models import RegressionModel
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from lightgbm import LGBMRegressor

def train_lightgbm_forecast(series, lags, forecast_horizon=1):
    """
    Train a LightGBM model using Darts' RegressionModel wrapper and forecast the next time steps.
    """
    # Create a TimeSeries object if not already
    if not isinstance(series, TimeSeries):
        series = TimeSeries.from_series(series)

    # Define the LightGBM model
    model = LGBMRegressor()

    # Darts RegressionModel wrapper around LightGBM
    reg_model = RegressionModel(
        model=model,
        lags=lags,  # specify lags: this can be a list of lags or an integer
        output_chunk_length=forecast_horizon
    )

    # Fit the model
    reg_model.fit(series)

    # Perform prediction
    forecast = reg_model.predict(n=forecast_horizon)

    return reg_model, forecast

# Example usage with a time series data
models = {}
forecasts = {}

# Assuming '_12_weekly_averages' is a DataFrame with time series data in each column
for column in _12_weekly_averages.columns:
    series = TimeSeries.from_series(_12_weekly_averages[column])
    model, forecast = train_lightgbm_forecast(series, lags=12)  # Assuming using past 12 periods as features
    models[column] = model
    forecasts[column] = forecast
    
    # Storing results
    actual = _12_weekly_averages[column][-1]  # last known value
    predicted = forecast.values()[-1][0]  # last predicted value
    df = pd.DataFrame([['LightGBM', column, actual, predicted, actual - predicted]], columns=['Model', 'Company', 'Actual', 'Forecasted', 'Error'])
    df_results = pd.concat([df_results, df])

df_results = df_results.reset_index(drop=True)

# Display results
print('Forecast results using LightGBM:')
print(df_results)


Forecast results using LightGBM:
         Model Company      Actual  Forecasted      Error
0           AR    AMZN   97.830002  101.222115  -3.392113
1           AR    GOOG  105.220001  100.845243   4.374758
2           AR    QCOM  120.269997  121.372908  -1.102911
3           AR    AMAT  113.160004  113.510634  -0.350630
4           MA    AMZN   97.830002  114.973522 -17.143520
5           MA    GOOG  105.220001   97.362835   7.857166
6           MA    QCOM  120.269997  123.432427  -3.162430
7           MA    AMAT  113.160004  106.906355   6.253649
8        ARIMA    AMZN   97.830002   97.797648   0.032354
9        ARIMA    GOOG  105.220001  103.297876   1.922125
10       ARIMA    QCOM  120.269997  121.961419  -1.691422
11       ARIMA    AMAT  113.160004  115.514663  -2.354659
12      SARIMA    AMZN   97.830002   98.360939  -0.530937
13      SARIMA    GOOG  105.220001  118.433662 -13.213661
14      SARIMA    QCOM  120.269997   89.036852  31.233145
15      SARIMA    AMAT  113.160004  138